# 기출2

In [1]:
import pandas as pd
import numpy as np

# !pip install mlxtend
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

from scipy.stats import chi2_contingency

dat=pd.read_csv('test/Train2.csv')
dat.head(5)

ID join_date sex  birth_year branch_code  P1  P2  P3  P4  P5  ...  \
0  4WKQSBB  1/2/2019   F        1987        1X1H   0   0   0   0   0  ...   
1  CP5S02H  1/6/2019   F        1981        UAOD   0   0   0   0   0  ...   
2  2YKDILJ  1/6/2013   M        1991        748L   0   0   0   0   0  ...   
3  2S9E81J  1/8/2019   M        1990        1X1H   0   0   0   0   0  ...   
4  BHDYVFT  1/8/2019   M        1990        748L   0   0   0   0   0  ...   

   P12  P13  P14  P15  P16  P17  P18  P19  P20  P21  
0    0    0    0    1    0    0    0    0    0    0  
1    0    0    0    1    0    0    0    0    0    0  
2    0    0    0    0    0    0    0    0    0    1  
3    0    0    0    1    0    0    0    0    0    0  
4    0    0    0    0    0    0    1    1    0    0  

[5 rows x 26 columns]

# 전처리

In [2]:
dat.join_date.isna().sum()

2

In [12]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29132 entries, 0 to 29131
Data columns (total 26 columns):
ID             29132 non-null object
join_date      29130 non-null object
sex            29132 non-null object
birth_year     29132 non-null int64
branch_code    29132 non-null object
P1             29132 non-null int64
P2             29132 non-null int64
P3             29132 non-null int64
P4             29132 non-null int64
P5             29132 non-null int64
P6             29132 non-null int64
P7             29132 non-null int64
P8             29132 non-null int64
P9             29132 non-null int64
P10            29132 non-null int64
P11            29132 non-null int64
P12            29132 non-null int64
P13            29132 non-null int64
P14            29132 non-null int64
P15            29132 non-null int64
P16            29132 non-null int64
P17            29132 non-null int64
P18            29132 non-null int64
P19            29132 non-null int64
P20            29132 no

In [14]:
pre1=dat.dropna(subset=['join_date']).reset_index(drop=True)

In [15]:
pre1['reg_age'] = pd.to_datetime(pre1['join_date']).dt.year - pre1['birth_year'] + 1
pre1.head(3)

ID join_date sex  birth_year branch_code  P1  P2  P3  P4  P5  ...  \
0  4WKQSBB  1/2/2019   F        1987        1X1H   0   0   0   0   0  ...   
1  CP5S02H  1/6/2019   F        1981        UAOD   0   0   0   0   0  ...   
2  2YKDILJ  1/6/2013   M        1991        748L   0   0   0   0   0  ...   

   P13  P14  P15  P16  P17  P18  P19  P20  P21  reg_age  
0    0    0    1    0    0    0    0    0    0       33  
1    0    0    1    0    0    0    0    0    0       39  
2    0    0    0    0    0    0    0    0    1       23  

[3 rows x 27 columns]

In [11]:
a1=pre1['join_date'].str.slice(-4).astype('int')
a1

0        2019
1        2019
2        2013
3        2019
4        2019
         ... 
29127    2019
29128    2018
29129    2019
29130    2019
29131    2020
Name: join_date, Length: 29130, dtype: int32

In [16]:
pre1.head(3)

ID join_date sex  birth_year branch_code  P1  P2  P3  P4  P5  ...  \
0  4WKQSBB  1/2/2019   F        1987        1X1H   0   0   0   0   0  ...   
1  CP5S02H  1/6/2019   F        1981        UAOD   0   0   0   0   0  ...   
2  2YKDILJ  1/6/2013   M        1991        748L   0   0   0   0   0  ...   

   P13  P14  P15  P16  P17  P18  P19  P20  P21  reg_age  
0    0    0    1    0    0    0    0    0    0       33  
1    0    0    1    0    0    0    0    0    0       39  
2    0    0    0    0    0    0    0    0    1       23  

[3 rows x 27 columns]

In [17]:
pre1['VIP'] = np.where(pre1.loc[:, 'P1':'P21'].sum(axis=1) >= 4,
                      'VIP', 'Not VIP')
pre1['VIP'].value_counts()

Not VIP    27900
VIP         1230
Name: VIP, dtype: int64

In [18]:
basetable1=pre1.copy()

# Q1

In [20]:
basetable1.groupby('branch_code')['reg_age'].mean().idxmax()

'49BM'

In [22]:
basetable1.groupby('branch_code')['reg_age'].mean().sort_values(ascending=False)

branch_code
49BM    42.745572
ZFER    40.631861
EU3L    40.222222
XX25    39.732620
BOAS    39.602305
O4JC    39.500000
E5SW    39.376138
748L    39.300184
30H5    39.258013
X23B    39.212121
9F9T    38.946387
94KC    38.857572
UAOD    38.736242
1X1H    38.534919
O67J    36.911032
Name: reg_age, dtype: float64

# Q2

In [24]:
q2=basetable1.loc[:,'P1':'P21']

In [25]:
q2.sum(axis=1).value_counts()

2     22731
3      5169
4       916
5       214
6        71
7        22
8         5
14        1
9         1
dtype: int64

In [23]:
q2=q2[q2.sum(axis=1) > 1].reset_index(drop=True)
len(basetable1), len(q2)

(29130, 29130)

In [27]:
te=apriori(q2, min_support=0.01, max_len=1, use_colnames=True)
te

support itemsets
0   0.061071     (P2)
1   0.010848     (P4)
2   0.011637     (P5)
3   0.014796     (P7)
4   0.869447     (P8)
5   0.074597     (P9)
6   0.012152    (P10)
7   0.028768    (P11)
8   0.018503    (P12)
9   0.011123    (P13)
10  0.742465    (P15)
11  0.234535    (P16)
12  0.010642    (P17)
13  0.048919    (P18)
14  0.048678    (P19)
15  0.030965    (P20)
16  0.037762    (P21)

In [29]:
te2=apriori(q2, min_support=0.01, max_len=2, use_colnames=True)
te2

support    itemsets
0   0.061071        (P2)
1   0.010848        (P4)
2   0.011637        (P5)
3   0.014796        (P7)
4   0.869447        (P8)
5   0.074597        (P9)
6   0.012152       (P10)
7   0.028768       (P11)
8   0.018503       (P12)
9   0.011123       (P13)
10  0.742465       (P15)
11  0.234535       (P16)
12  0.010642       (P17)
13  0.048919       (P18)
14  0.048678       (P19)
15  0.030965       (P20)
16  0.037762       (P21)
17  0.035736    (P2, P8)
18  0.013663   (P15, P2)
19  0.016547   (P2, P16)
20  0.036629    (P9, P8)
21  0.742465   (P15, P8)
22  0.177686   (P16, P8)
23  0.010642   (P17, P8)
24  0.012530   (P18, P8)
25  0.012461   (P19, P8)
26  0.019739   (P20, P8)
27  0.033505   (P21, P8)
28  0.014864   (P15, P9)
29  0.034672   (P9, P16)
30  0.016547  (P12, P11)
31  0.117027  (P15, P16)
32  0.010333  (P15, P17)
33  0.014178  (P15, P20)
34  0.020460  (P15, P21)
35  0.012873  (P18, P16)
36  0.012702  (P19, P16)
37  0.015963  (P16, P20)
38  0.048678  (P18, P19)

In [37]:
q3=association_rules(te2,metric='confidence', min_threshold=0.01)
q3

antecedents consequents  antecedent support  consequent support   support  \
0         (P2)        (P8)            0.061071            0.869447  0.035736   
1         (P8)        (P2)            0.869447            0.061071  0.035736   
2        (P15)        (P2)            0.742465            0.061071  0.013663   
3         (P2)       (P15)            0.061071            0.742465  0.013663   
4         (P2)       (P16)            0.061071            0.234535  0.016547   
5        (P16)        (P2)            0.234535            0.061071  0.016547   
6         (P9)        (P8)            0.074597            0.869447  0.036629   
7         (P8)        (P9)            0.869447            0.074597  0.036629   
8        (P15)        (P8)            0.742465            0.869447  0.742465   
9         (P8)       (P15)            0.869447            0.742465  0.742465   
10       (P16)        (P8)            0.234535            0.869447  0.177686   
11        (P8)       (P16)            0.869447            0.234535  0.177686   
12       (P17)        (P8)            0.010642            0.869447  0.010642   
13        (P8)       (P17)            0.869447            0.010642  0.010642   
14       (P18)        (P8)            0.048919            0.869447  0.012530   
15        (P8)       (P18)            0.869447            0.048919  0.012530   
16       (P19)        (P8)            0.048678            0.869447  0.012461   
17        (P8)       (P19)            0.869447            0.048678  0.012461   
18       (P20)        (P8)            0.030965            0.869447  0.019739   
19        (P8)       (P20)            0.869447            0.030965  0.019739   
20       (P21)        (P8)            0.037762            0.869447  0.033505   
21        (P8)       (P21)            0.869447            0.037762  0.033505   
22       (P15)        (P9)            0.742465            0.074597  0.014864   
23        (P9)       (P15)            0.074597            0.742465  0.014864   
24        (P9)       (P16)            0.074597            0.234535  0.034672   
25       (P16)        (P9)            0.234535            0.074597  0.034672   
26       (P12)       (P11)            0.018503            0.028768  0.016547   
27       (P11)       (P12)            0.028768            0.018503  0.016547   
28       (P15)       (P16)            0.742465            0.234535  0.117027   
29       (P16)       (P15)            0.234535            0.742465  0.117027   
30       (P15)       (P17)            0.742465            0.010642  0.010333   
31       (P17)       (P15)            0.010642            0.742465  0.010333   
32       (P15)       (P20)            0.742465            0.030965  0.014178   
33       (P20)       (P15)            0.030965            0.742465  0.014178   
34       (P15)       (P21)            0.742465            0.037762  0.020460   
35       (P21)       (P15)            0.037762            0.742465  0.020460   
36       (P18)       (P16)            0.048919            0.234535  0.012873   
37       (P16)       (P18)            0.234535            0.048919  0.012873   
38       (P19)       (P16)            0.048678            0.234535  0.012702   
39       (P16)       (P19)            0.234535            0.048678  0.012702   
40       (P16)       (P20)            0.234535            0.030965  0.015963   
41       (P20)       (P16)            0.030965            0.234535  0.015963   
42       (P18)       (P19)            0.048919            0.048678  0.048678   
43       (P19)       (P18)            0.048678            0.048919  0.048678   

    confidence       lift  leverage  conviction  
0     0.585160   0.673025 -0.017362    0.314706  
1     0.041102   0.673025 -0.017362    0.979175  
2     0.018402   0.301322 -0.031680    0.956531  
3     0.223721   0.301322 -0.031680    0.331756  
4     0.270939   1.155217  0.002223    1.049933  
5     0.070550   1.155217  0.002223    1.010199  
6     0.491026   0.564757 -0.028229    0.256502  
7 

In [34]:
len(q3)

8

In [38]:
q3['antecedents'].head(2)

0    (P2)
1    (P8)
Name: antecedents, dtype: object

In [45]:
q3_2=q3[q3['antecedents']=={'P15'}]
q3_2.sort_values(by='lift', ascending=False).head(1)['lift']

30    1.307763
Name: lift, dtype: float64

In [46]:
q3.head()

antecedents consequents  antecedent support  consequent support   support  \
0        (P2)        (P8)            0.061071            0.869447  0.035736   
1        (P8)        (P2)            0.869447            0.061071  0.035736   
2       (P15)        (P2)            0.742465            0.061071  0.013663   
3        (P2)       (P15)            0.061071            0.742465  0.013663   
4        (P2)       (P16)            0.061071            0.234535  0.016547   

   confidence      lift  leverage  conviction  
0    0.585160  0.673025 -0.017362    0.314706  
1    0.041102  0.673025 -0.017362    0.979175  
2    0.018402  0.301322 -0.031680    0.956531  
3    0.223721  0.301322 -0.031680    0.331756  
4    0.270939  1.155217  0.002223    1.049933

In [47]:
q2

P1  P2  P3  P4  P5  P6  P7  P8  P9  P10  ...  P12  P13  P14  P15  P16  \
0       0   0   0   0   0   0   0   1   0    0  ...    0    0    0    1    0   
1       0   0   0   0   0   0   0   1   0    0  ...    0    0    0    1    0   
2       0   0   0   0   0   0   1   1   0    0  ...    0    0    0    0    0   
3       0   0   0   0   0   0   0   1   0    0  ...    0    0    0    1    0   
4       0   0   0   0   0   0   0   0   0    0  ...    0    0    0    0    0   
...    ..  ..  ..  ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...  ...  ...   
29125   0   0   0   0   0   0   0   1   0    0  ...    0    0    0    1    0   
29126   0   0   0   0   0   0   0   1   0    0  ...    0    0    0    1    0   
29127   0   0   0   0   0   0   0   1   0    0  ...    0    0    0    1    0   
29128   0   0   0   0   0   0   0   1   0    0  ...    0    0    0    1    0   
29129   0   0   0   0   0   0   0   1   0    0  ...    0    0    0    1    0   

       P17  P18  P19  P20  P21  
0        0    0    0    0    0  
1        0    0    0    0    0  
2        0    0    0    0    1  
3        0    0    0    0    0  
4        0    1    1    0    0  
...    ...  ...  ...  ...  ...  
29125    0    0    0    0    0  
29126    0    0    0    0    0  
29127    0    0    0    0    0  
29128    0    0    0    0    0  
29129    0    0    0    0    0  

[29130 rows x 21 columns]

In [50]:
q2_tf=q2.replace({1: True, 0:False})

In [51]:
q2_tf.columns[q2_tf.iloc[0,:]]

Index(['P8', 'P15'], dtype='object')

In [55]:
q2_tf.apply(lambda x: list(q2_tf.columns[x]), axis=1)

0            [P8, P15]
1            [P8, P15]
2        [P7, P8, P21]
3            [P8, P15]
4           [P18, P19]
             ...      
29125        [P8, P15]
29126        [P8, P15]
29127        [P8, P15]
29128        [P8, P15]
29129        [P8, P15]
Length: 29130, dtype: object

In [52]:
q2.columns.isin(['P2', 'P21'])

array([False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False,  True])

In [53]:
# q2[조건(행방향)]
# q2.loc[행방향조건, 열방향조건]

q2.loc[:, q2.columns.isin(['P2', 'P21'])]

P2  P21
0       0    0
1       0    0
2       0    1
3       0    0
4       0    0
...    ..  ...
29125   0    0
29126   0    0
29127   0    0
29128   0    0
29129   0    0

[29130 rows x 2 columns]

# Q3

In [57]:
tab=pd.crosstab(basetable1.VIP, basetable1.sex)
q3=chi2_contingency(tab)
q3[1]

0.04774121750605259